# Caravan and Camping Club POI Generator 2021

In [1]:
# Install any missing libraries
!pip install simplekml
!pip install pandas
!pip install folium

In [1]:
# imports
import pandas as pd
import simplekml
import folium

In [2]:
df = pd.read_csv("data/Listed-Sites.csv",names=['Longitude','Latitude','Address'], header=0, sep=',')
print(df.head(5))
print(df['Address'].head())

   Longitude   Latitude                                            Address
0  -3.856600  56.624500  "Aberfeldy - Aberfeldy Caravan Site PH15 2AQ>0...
1  -4.354700  56.139700  "Aberfoyle - Trossachs Holiday Park FK8 3SA>01...
2  -3.631834  53.262374  "Abergele - Bryn Ffanigl Ganol LL22 8NF>01492 ...
3  -3.605300  53.248300  "Abergele - Hunters Hamlet Caravan Park LL22 8...
4  -3.596100  53.282600  "Abergele - Manorafon Farm Touring & Camping L...
0    "Aberfeldy - Aberfeldy Caravan Site PH15 2AQ>0...
1    "Aberfoyle - Trossachs Holiday Park FK8 3SA>01...
2    "Abergele - Bryn Ffanigl Ganol LL22 8NF>01492 ...
3    "Abergele - Hunters Hamlet Caravan Park LL22 8...
4    "Abergele - Manorafon Farm Touring & Camping L...
Name: Address, dtype: object


### Extract Site Name, Telephone Number, and Address in to their own columns

In [3]:
temp = df['Address'].apply(lambda x: pd.Series(str(x).split('>')))
print(temp)  # check if data is okay, one line had less quotes in it for some reason!!

df["PhoneNumber"]= temp[1]
df['PhoneNumber'] = df['PhoneNumber'].astype(str)

# Remove extra quotes the camping club add to the end of each line!
df['PhoneNumber'] = df.apply(lambda x: x['PhoneNumber'][:-2], axis=1)

# making separate phone number column from new data frame 
temp_address = temp[0].apply(lambda x: pd.Series(str(x).split(' - ')))
df["Town"] = temp_address[0]
df["SiteName"] = temp_address[1]

df["Town"] = df.apply(lambda x: x['Town'][1:], axis=1)

# Dropping old Address columns 
df.drop(columns =["Address"], inplace = True) 

df 

                                                     0               1
0         "Aberfeldy - Aberfeldy Caravan Site PH15 2AQ  01887 822108""
1          "Aberfoyle - Trossachs Holiday Park FK8 3SA  01877 382614""
2              "Abergele - Bryn Ffanigl Ganol LL22 8NF  01492 680274""
3     "Abergele - Hunters Hamlet Caravan Park LL22 8PL  01745 832237""
4    "Abergele - Manorafon Farm Touring & Camping L...  01745 833237""
..                                                 ...             ...
804  "York - Grantchester Caravan and Camping YO30 1YJ  07472 694380""
805            "York - Nurseries Caravan Park YO23 3QY  01904 797427""
806                 "York - York Caravan Park YO32 9UB  01904 424222""
807   "Youlgreave - Hopping Farm Caravan park DE45 1NA  01629 636302""
808  "Ystrad Aeron - Hafod Brynog Caravan Park SA48...  01570 470084""

[809 rows x 2 columns]


,Longitude,Latitude,PhoneNumber,Town,SiteName
0,-3.856600,56.624500,01887 822108,Aberfeldy,Aberfeldy Caravan Site PH15 2AQ
1,-4.354700,56.139700,01877 382614,Aberfoyle,Trossachs Holiday Park FK8 3SA
2,-3.631834,53.262374,01492 680274,Abergele,Bryn Ffanigl Ganol LL22 8NF
3,-3.605300,53.248300,01745 832237,Abergele,Hunters Hamlet Caravan Park LL22 8PL
4,-3.596100,53.282600,01745 833237,Abergele,Manorafon Farm Touring & Camping LL22 8ET
...,...,...,...,...,...
804,-1.137307,53.998185,07472 694380,York,Grantchester Caravan and Camping YO30 1YJ
805,-1.146469,53.936300,01904 797427,York,Nurseries Caravan Park YO23 3QY
806,-1.030800,53.982100,01904 424222,York,York Caravan Park YO32 9UB
807,-1.687400,53.164300,01629 636302,Youlgreave,Hopping Farm Caravan park DE45 1NA


### Now create kml file for use by maps.me

### Once the .kml file has been created for maps.me will need to edit the file and change all instances of <styleUrl>#<number></styleUrl> with a valid maps.me styleUrl
    
<styleUrl>#placemark-blue</styleUrl>
<styleUrl>#placemark-red</styleUrl>
<styleUrl>#placemark-green</styleUrl>  
<styleUrl>#placemark-yellow</styleUrl>
<styleUrl>#placemark-brown</styleUrl>
<styleUrl>#placemark-orange</styleUrl>
<styleUrl>#placemark-pink</styleUrl>    
<styleUrl>#placemark-purple</styleUrl> *

In [4]:
kml = simplekml.Kml()
style = simplekml.Style()
style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/ltblu-blank.png'
style.iconstyle.scale = 1

map = folium.Map(location=[54.0, -1.0], zoom_start=6)
for index, row in df.iterrows():
    #name = row['SiteName'] + ', ' + row['SiteAddress'] + ', ' + row['PhoneNumber']
    name = row['SiteName']
    address = row['Town'] + ', ' + row['SiteName'] + ', ' + row['PhoneNumber']
    phonenumber = row['PhoneNumber']
    coords = [(row['Longitude'], row['Latitude'])]
    pnt = kml.newpoint(name=name)
    pnt.coords = coords
    pnt.style = style
    pnt.phonenumber = phonenumber
    pnt.description = address
    
    popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+'</p><p>'+row['PhoneNumber']+'</p>', script=True)
    #popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p>', script=True)
    popup = folium.Popup(popupText, max_width=300)
    folium.Marker([row['Latitude'], row['Longitude']],
                 popup=popup,
                 icon=folium.Icon(color='lightblue', icon='info-sign')).add_to(map)
    
kml.save(path="poi/ccc-listed-2021.kml")
map.save("poi/ccc-listed-2021.html")
map

In [46]:
#!pip list